In [ ]:
# El contenido con mayor cantidad de de acciones realizadas para todos los tipos posibles de acciones (⭐⭐⭐).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
import re as regex

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
rdd_logs = spark.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/logs.csv', header=True, inferSchema=True, multiLine=True, escape='"').rdd

In [ ]:
rdd_logs.first()

Row(item_id=1, timestamp=datetime.datetime(2004, 12, 23, 7, 10, 9), contributor_username='Sabbut', contributor_id=2075, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo')

In [ ]:
rdd_logs_cached = rdd_logs.filter(lambda x:x.contributor_id!=None and x.action!=None and x.title!=None and ":" not in x.title).cache()

In [ ]:
rdd_logs_cached = rdd_logs_cached.map(lambda x:  ((x.title,x.action), 1)  ).reduceByKey(lambda a,b: a+b).cache()

In [ ]:
rdd_logs_cached.map(lambda x:( x[0][1], (x[1], x[0][0]) ) ).take(10)

[('delete', (1, 'Autovalvula')),
 ('delete', (1, 'MIDI')),
 ('delete', (7, 'Socialismo científico')),
 ('delete', (1, 'WMV')),
 ('delete', (1, 'Wight')),
 ('delete', (1, 'Gran Telescopio Canarias')),
 ('delete', (1, 'Adeodato I')),
 ('delete', (1, 'Gestion del Conocimiento')),
 ('delete', (1, 'Dispensar')),
 ('delete', (2, 'Juicio de Paris'))]

In [ ]:
rdd_final = rdd_logs_cached.map(lambda x:( x[0][1], (x[1], x[0][0]) ) )

Respuesta: El contenido con más acciones realizadas para cada accion. ('Action' , (# acciones, titulo del contenido)

In [ ]:
rdd_final.reduceByKey(lambda a,b: a if a[0] > b[0] else b).collect()

[('delete', (77, 'Agarradera (química)')),
 ('move', (14, 'La Concordia (Nicaragua)')),
 ('move_redir', (15, 'Suspenso')),
 ('protect', (37, 'José de San Martín')),
 ('restore', (15, 'Abel Azcona')),
 ('unprotect', (12, 'Cataluña')),
 ('patrol', (4, 'Yoshitoshi')),
 ('modify', (23, 'Cristina Fernández de Kirchner')),
 ('move_prot', (9, 'Rebelión en Libia de 2011')),
 ('revision', (43, 'Detención de Milagro Sala')),
 ('add', (57, '(commons)')),
 ('update', (113, 'Bandera del Reino Unido')),
 ('selfadd', (15, 'Ingeniería de software')),
 ('delete_redir', (11, 'Negre Lloma')),
 ('remove', (3, 'Hot Pockets')),
 ('create', (12, 'Charlotte Caniggia')),
 ('selfremove', (3, 'Software')),
 ('review', (13, 'Ingeniería de software')),
 ('addlink', (1, 'Ethel Spears')),
 ('unreview', (4, 'San Ángel')),
 ('renameuser', (1, 'Spain')),
 ('merge', (2, 'Amortiguación')),
 ('upload', (2, 'Lobo hombre en París')),
 ('skipbadns', (14, 'User Calypso')),
 ('interwiki', (2, 'Actinozoa'))]